## Documentação Executiva: Projeto Delivery Unit Economics

Este documento centraliza o contexto de negócio, a arquitetura final e os resultados do pipeline ETL modularizado que calcula a Unit Economics (UE) dos pedidos de delivery.

### 1. Contexto de Negócio e Objetivo Estratégico

| Categoria | Descrição |
| :--- | :--- |
| Objetivo | Criar uma visão unificada e calculada do pedido para determinar o Lucro Bruto Unitário (Margem de Contribuição) por transação. |
| Desafio | Unir dados dispersos de Pedidos, Pagamentos e Logística, aplicando regras de deduplicação e Unit Economics para gerar uma métrica de P&L coerente. |
| Impacto | Essencial para otimizar preços, comissões e repasses (P&L), garantindo a sustentabilidade da plataforma Two-Sided. |

### 2. Arquitetura Final: Modelo Medalhão (Bronze, Silver, Gold)

O projeto segue a arquitetura Delta Lakehouse em três camadas, garantindo qualidade e modularidade.

| Camada | Objetivo | Componentes Chave |
| :--- | :--- | :--- |
| Bronze (Cru) | Ingestão e armazenamento de dados brutos (Full Load). | tbl_fact_orders_bronze, tbl_dim_stores_bronze, etc. |
| Silver (Refinado) | Cálculo da Unit Economics (UE). Aplica regras de Deduplicação, Tipagem e Validação de Nulos/Extremos. | tbl_fact_pedidos_silver (Fato Central). |
| Gold (Consumo) | Modelagem dimensional (Star Schema) pronta para Business Intelligence (Power BI). | tbl_fato_delivery_gold, tbl_dim_store_gold, tbl_dim_time_gold. |

### 3. Etapas do Pipeline (Workflow de Execução)

O processo de ETL é executado em um único Job do Databricks, garantindo dependências e tratamento de erros (try/except).

| # | Notebook | Função Principal | Status de Log |
| :--- | :--- | :--- | :--- |
| 01 | 00_Setup_Logging.ipynb | Criação do Schema e tabelas de log (log_processamento, metricas_qualidade). | pipeline_setup |
| 02 | 01_01_bronze_ingestion.ipynb | Ingestão dos dados brutos (Full Load) e padronização inicial dos nomes de colunas. | bronze_ingestion |
| 03 | 01_02_silver_ue_calculation_dq.ipynb | Processa Bronze, deduplica (1:1), calcula as 9 métricas da UE e aplica DQ de Negócio (nulos, negativos). |silver_ue_calculation |
| 04 | 01_03_gold_model.ipynb | Cria o Star Schema Gold (Fato + Dimensões) e otimiza com ZORDER. | gold_dimensional_model |
| 05 | 04_Final_Quality_Checks.ipynb | Executa Data Testing de alta criticidade (Integridade Referencial e Validação de Constantes). | final_data_testing |

### 4. Resultados e Governança de Dados
O projeto entrega um conjunto de dados limpo e validado, com forte governança sobre as regras de negócio.

#### A. Resultados Financeiros (Amostra da Tabela Silver)

Total de Registros Limpos (Fatos Gold): 347.567 (Pós-deduplicação e filtro de DELIVERED).

Margem Média: [Valor calculado na Silver]

Pedidos Não-Rentáveis: [Percentual calculado na Silver]

#### B. Sucesso dos Testes de Qualidade (Governança)

Os testes de Data Testing foram executados com sucesso, garantindo a integridade dos dados prontos para o BI:

| Teste de Governança | Status (Valor) | Confirmação |
| :--- | :--- | :--- |
| Integridade Referencial (store_id Gold) | 0.00 violações | Todo pedido possui uma loja válida. |
| Validação de Constantes (UE) | 0.00 inconsistências | A lógica de cálculo da UE está 100% consistente com as premissas de negócio. |

### 5. Benefícios da Arquitetura Modularizada

- Confiabilidade de Dados (DQ): Implementação de Data Testing completo, garantindo a integridade referencial e a estabilidade do cálculo da UE.

- Consistência 1:1 Resolvida: Resolução do problema de duplicação (Fan-out), garantindo que cada métrica seja calculada para um único pedido.

- Manutenibilidade: Código dividido em etapas lógicas e isoladas, facilitando o debugging e a inclusão de novas regras de negócio.

- Performance Otimizada: Criação de tabelas Gold dimensionais otimizadas (ZORDER) para consumo rápido no Power BI.

### STATUS FINAL DO PROJETO: PRODUCTION READY